In [2]:
import requests
import json

In [3]:
API_KEY = 'RGAPI-5b1aae35-12e5-4c9c-bc40-64b7acdd558b'
FEATURED_GAMES_URL = f'https://na1.api.riotgames.com/lol/spectator/v4/featured-games?api_key={API_KEY}'
SUMMONER_INFO_URL = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
MATCH_ID_URL = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
MATCH_INFO_URL = 'https://americas.api.riotgames.com/lol/match/v5/matches/'

In [4]:
# getting a list of currently featured games
games_req = requests.get(FEATURED_GAMES_URL).text
all_games = json.loads(games_req)

# gathering names of all players in each featured game
summoners = []
for game in all_games['gameList']:
    for particpant in game['participants']:
        summoners.append(particpant['summonerName'])

# gathering the encrypted puuids of each suummoner and then finding the match_ids of their first 20 recent matches.
match_ids = []
for summoner in summoners:
    url = f'{SUMMONER_INFO_URL}{summoner}?api_key={API_KEY}'
    summoner_req = requests.get(url).text
    summoner_info = json.loads(summoner_req)

    COUNT = 20
    url = f'{MATCH_ID_URL}{summoner_info["puuid"]}/ids?start=0&count={COUNT}&api_key={API_KEY}'
    match_ids_req = requests.get(url).text
    matches = json.loads(match_ids_req)

    match_ids += matches

# Saving the match IDs into a text file to keep for further analysis/use
with open("saved-match-ids.txt", "a") as OUTPUT_FILE:
    print(match_ids, file=OUTPUT_FILE)


In [7]:
url = f'{MATCH_INFO_URL}{match_ids[0]}?api_key={API_KEY}'
match_info_req = requests.get(url).text

info = json.loads(match_info_req)
print(json.dumps(info['info'], indent=2))



{
  "gameCreation": 1652051720000,
  "gameDuration": 947,
  "gameEndTimestamp": 1652052704104,
  "gameId": 4301360265,
  "gameMode": "ARAM",
  "gameName": "teambuilder-match-4301360265",
  "gameStartTimestamp": 1652051756149,
  "gameType": "MATCHED_GAME",
  "gameVersion": "12.8.437.6765",
  "mapId": 12,
  "participants": [
    {
      "assists": 21,
      "baronKills": 0,
      "bountyLevel": 0,
      "challenges": {
        "12AssistStreakCount": 0,
        "abilityUses": 145,
        "acesBefore15Minutes": 2,
        "alliedJungleMonsterKills": 0,
        "baronTakedowns": 0,
        "blastConeOppositeOpponentCount": 0,
        "bountyGold": 300,
        "buffsStolen": 0,
        "completeSupportQuestInTime": 0,
        "controlWardsPlaced": 0,
        "damagePerMinute": 1210.952258634774,
        "damageTakenOnTeamPercentage": 0.17493158576617315,
        "dancedWithRiftHerald": 0,
        "deathsByEnemyChamps": 6,
        "dodgeSkillShotsSmallWindow": 31,
        "doubleAces": 0,
 